# Exception Rates for New Scarlet

Count up the exception rates from the logs. First run analysis for state when we first got the deblender running. Scroll to bottom for end of sprint rerun after some exception fixes.

## Data

Used the run in:

__`/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym/logs`__


## Stack Version
This was produced with the following versions:
```
lsst_distrib 17.0+52      current w_2019_19 setup

autograd              LOCAL:/project/yusra/lsst_devel/autograd/_eupspkg/binary/autograd/master-g6a9ef8c4bf      setup
meas_extensions_scarlet LOCAL:/project/yusra/lsst_devel/meas_extensions_scarlet         setup
pipe_drivers          LOCAL:/project/yusra/lsst_devel/pipe_drivers      setup
pipe_tasks            LOCAL:/project/yusra/lsst_devel/pipe_tasks        setup
scarlet               LOCAL:/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3    setup
```

- *where pipe_tasks has is on DM-19720:*
```
commit aa5e78b53f2bac60691a206f0688554dfb74b06c
Author: Nate Lust <nlust@astro.princeton.edu>
Date:   Mon May 13 08:57:20 2019 -0500

    Fix deblender logging
```


- *pipe_drivers is on DM-19720:*
```
commit f880519c69f5ebe1ae0d6632a95167e579490c9b
Author: Yusra AlSayyad <yusra@astro.princeton.edu>
Date:   Mon May 13 15:58:24 2019 -0500

    Do not check for
    
    which is not a config parameter in ScarletDeblendTask
```


- *meas_extensions_scarlet is on DM-19720 with a cherry-pick of Fred’s latest commit `git cherry-pick 136e4054f24572e1ba3d1566f5cfbe85c1218b53` on `deblender_sprint`. The sha changed because it required a merge-fix:*

```
commit 9c6da290824cb3b9b9d609a81ec360e27939a7c9
Author: fred3m <fredem@princeton.edu>
Date:   Mon May 13 14:44:30 2019 -0500

    Update with scarlet master

commit afa31966330fd7c6ac77b2bcc522de69a76ca6da
Author: Nate Lust <nlust@astro.princeton.edu>
Date:   Mon May 13 12:27:35 2019 -0500

    Properly pass arguments to Source init
```

Example command to produce output repo:

```basj
multiBandDriver.py /datasets/hsc/repo --calib /datasets/hsc/repo/CALIB/ --rerun private/yusra/deblender-sprint/w_2019_18_05k:private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym --batch-type=slurm --mpiexec='-bind-to socket' --job tiny3 --nodes 5 --procs 4 --time 3000 --id tract=9813 patch=$TINYEXISTS3 filter=HSC-G^HSC-R^HSC-I^HSC-Z^HSC-Y^NB0921 --config deblendCoaddSources.simultaneous=True  deblendCoaddSources.multiBandDeblend.catchFailures=True measureCoaddSources.inputCatalog=deblendedModel deblendCoaddSources.multiBandDeblend.symmetric=False --reuse-outputs-from all
```

## Extract Deblend Attempts and Failures

Extract lines like (for example):
```bash
grep "Unable to deblend" tiny4.o154140 >> UnableToDeblendLines
grep "Unable to deblend" tiny3.o154141 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154139 >> UnableToDeblendLines
grep "Unable to deblend" tiny4.o154138 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154137 >> UnableToDeblendLines
grep "Unable to deblend" tiny6.o154135 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154131 >> UnableToDeblendLines
grep "Unable to deblend" tiny4.o154130 >> UnableToDeblendLines
grep "Unable to deblend" tiny3.o154129 >> UnableToDeblendLines

grep "meas.deblender.deblend: id:" tiny4.o154140 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny3.o154141 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154139 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny4.o154138 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154137 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny6.o154135 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154131 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny4.o154130 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny3.o154129 >> DeblendAttemptLines
```

__OR__ if you can describe the logs with a wildcard string:

```bash
grep -s "Unable to deblend" tiny*.o* > UnableToDeblendLines
grep -s "meas.deblender.deblend: id:" tiny*.o* > DeblendAttemptLines
```

Note some of those logs are restarted failed reruns and the same blend id may appear more than once. These are removed after reading into a DataFrame


## Results:

In [1]:
import pandas as pd
import numpy as np
import os

DATA_DIR = '/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym/logs'

df = pd.DataFrame.from_csv(os.path.join(DATA_DIR, 'UnableToDeblendLines'),  sep=': ', header=None)
df = df.drop_duplicates() 
attempted = pd.DataFrame.from_csv(os.path.join(DATA_DIR, 'DeblendAttemptLines'),  sep=': ', header=None)
attempted = attempted.drop_duplicates()

/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/pandas/core/frame.py:1901: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  infer_datetime_format=infer_datetime_format)
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:9: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments 

In [2]:
groups = df.groupby(df[2])
groups.count()

,1
2,
'LsstSource' object has no attribute 'B',70
attempt to get argmax of an empty sequence,51
"could not broadcast input array from shape (41,41) into shape (41)",425
"could not broadcast input array from shape (41,43) into shape (41)",452
"could not broadcast input array from shape (43,41) into shape (43)",428
"could not broadcast input array from shape (43,43) into shape (43)",1789
tuple index out of range,2954


In [3]:
print("total attempted:", len(attempted))
print("number with exceptions:", len(df[2]))
print("fraction with exceptions:", len(df[2])/len(attempted))

total attempted: 49447
number with exceptions: 6185
fraction with exceptions: 0.12508342265455943


# Match to Flags

In [4]:
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler('/project/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym')

In [5]:
blendDF = pd.DataFrame()

blendList = []
patchList = []
tinyPatchList = ['{},{}'.format(i, j) for i in range(15, 32) for j in range(8, 32)]
for patchStr in tinyPatchList:
    #filter doesn't matter
    dataId = {'tract': 9813, 'filter': 'HSC-I', 'patch': patchStr}
    filename = butler.get("deepCoadd_deblendedModel_filename", dataId=dataId)[0]
    if os.path.exists(filename):
        src = butler.get("deepCoadd_deblendedModel", dataId=dataId)
        src = src.asAstropy().to_pandas()
        src['patch'] = patchStr
        blendList.append(src)

blendDF = pd.concat(blendList)

In [6]:
df.columns = ['error', 'message']
df['id'] = df['error'].apply(lambda x: x.split(' ')[-1].replace(':','')).astype('int64')
merged = pd.merge(df, blendDF, left_on='id', right_on='id', how='left')

In [7]:
mask = merged['message'].apply(lambda x: x is None)
merged.loc[mask,'message'] = ''

In [8]:
merged['message_type'] = merged['message'].apply(lambda x: 'could not broadcast input array' if x.startswith('could') else x)
merged['outcome'] = merged['deblend_failed'].apply(lambda x: 'flagged failed' if x == True else ('not in catalog' if np.isnan(x) else 'not flagged'))

In [9]:
merged.groupby([merged.message_type, merged.outcome]).count()

error  message  \
message_type                               outcome                          
                                           flagged failed     15       15   
                                           not flagged         1        1   
'LsstSource' object has no attribute 'B'   flagged failed     70       70   
attempt to get argmax of an empty sequence flagged failed     51       51   
could not broadcast input array            flagged failed   3094     3094   
tuple index out of range                   flagged failed   2954     2954   

                                                             id  coord_ra  \
message_type                               outcome                          
                                           flagged failed    15         0   
                                           not flagged        1         0   
'LsstSource' object has no attribute 'B'   flagged failed    70         0   
attempt to get argmax of an empty sequence flagged failed    51         0   
could not broadcast input array            flagged failed  3094         0   
tuple index out of range                   flagged failed  2954         0   

                                                           coord_dec  parent  \
message_type                               outcome                             
                                           flagged failed          0      15   
                                           not flagged             0       1   
'LsstSource' object has no attribute 'B'   flagged failed          0      70   
attempt to get argmax of an empty sequence flagged failed          0      51   
could not broadcast input array            flagged failed          0    3094   
tuple index out of range                   flagged failed          0    2954   

                                                           merge_footprint_i2  \
message_type                               outcome                              
                                           flagged failed                  15   
                                           not flagged                      1   
'LsstSource' object has no attribute 'B'   flagged failed                  70   
attempt to get argmax of an empty sequence flagged failed                  51   
could not broadcast input array            flagged failed                3094   
tuple index out of range                   flagged failed                2954   

                                                           merge_footprint_i  \
message_type                               outcome                             
                                           flagged failed                 15   
                                           not flagged                     1   
'LsstSource' object has no attribute 'B'   flagged failed                 70   
attempt to get argmax of an empty sequence flagged failed                 51   
could not broadcast input array            flagged failed               3094   
tuple index out of range                   flagged failed               2954   

                                                           merge_footprint_r2  \
message_type                               outcome                              
                                           flagged failed                  15   
                                           not flagged                      1   
'LsstSource' object has no attribute 'B'   flagged failed                  70   
attempt to get argmax of an empty sequence flagged failed                  51   
could not broadcast input array            flagged failed                3094   
tuple index out of range                   flagged failed                2954   

                                                           merge_footprint_r  \
message_type                               outcome                             
                                           flagged failed                 15   
     

In [10]:
merged[(merged.outcome != "flagged failed")]

,error,message,id,coord_ra,coord_dec,parent,merge_footprint_i2,merge_footprint_i,merge_footprint_r2,merge_footprint_r,...,deblend_masked,deblend_sedConvergenceFailed,deblend_morphConvergenceFailed,deblend_blendConvergenceFailed,deblend_edgePixels,deblend_failed,deblend_skipped,patch,message_type,outcome
2069,Unable to deblend source 43160714768023580:,,43160714768023580,NaN,NaN,0,False,True,False,True,...,False,False,False,True,False,False,False,"19,17",,not flagged


In [11]:
print("Counts")
merged['id'].apply(bool).groupby([merged.message_type, merged.outcome]).sum()

Counts


message_type                                outcome       
                                            flagged failed      15.0
                                            not flagged          1.0
'LsstSource' object has no attribute 'B'    flagged failed      70.0
attempt to get argmax of an empty sequence  flagged failed      51.0
could not broadcast input array             flagged failed    3094.0
tuple index out of range                    flagged failed    2954.0
Name: id, dtype: float64

In [12]:
print("Fraction")
merged['id'].apply(bool).groupby([merged.message_type, merged.outcome]).sum()/len(attempted)

Fraction


message_type                                outcome       
                                            flagged failed    0.000303
                                            not flagged       0.000020
'LsstSource' object has no attribute 'B'    flagged failed    0.001416
attempt to get argmax of an empty sequence  flagged failed    0.001031
could not broadcast input array             flagged failed    0.062572
tuple index out of range                    flagged failed    0.059741
Name: id, dtype: float64

# Conclusions (Beginning of Sprint)

* 12% fail with an exception
* Some exceptions aren't in the output catalogs at all


# Update 5/16 with Demo repo

A few of the exception types were fixed in the run:
__`/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym_demo/logs`__

```bash
grep -s "Unable to deblend"  TINY[0-4].o* > UnableToDeblendLines
grep -s "Unable to deblend"  TINY[1-4]_2.o* >> UnableToDeblendLines
grep -s "meas.deblender.deblend: id:" TINY[0-4].o* > DeblendAttemptLines
grep -s "meas.deblender.deblend: id:" TINY[1-4]_2.o* >> DeblendAttemptLines
```

In [13]:
AFTER_DATA_DIR = "/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym_demo"

df = pd.DataFrame.from_csv(os.path.join(AFTER_DATA_DIR, 'logs', 'UnableToDeblendLines'),  sep=': ', header=None)
df = df.drop_duplicates() 
attempted = pd.DataFrame.from_csv(os.path.join(AFTER_DATA_DIR, 'logs', 'DeblendAttemptLines'),  sep=': ', header=None)
attempted = attempted.drop_duplicates()

butlerDemo = dafPersist.Butler(AFTER_DATA_DIR)
blendList = []
patchList = []
tinyPatchList = ['{},{}'.format(i, j) for i in range(15, 32) for j in range(8, 32)]
for patchStr in tinyPatchList:
    #filter doesn't matter
    dataId = {'tract': 9813, 'filter': 'HSC-I', 'patch': patchStr}
    filename = butlerDemo.get("deepCoadd_deblendedModel_filename", dataId=dataId)[0]
    if os.path.exists(filename):
        src = butlerDemo.get("deepCoadd_deblendedModel", dataId=dataId)
        src = src.asAstropy().to_pandas()
        blendList.append(src)
        patchList.append(patchStr)
        
blendDF = pd.concat(blendList)

/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  app.launch_new_instance()
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/pandas/core/frame.py:1901: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  infer_datetime_format=infer_datetime_format)
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that so

In [14]:
df.columns = ['error', 'message']
df['id'] = df['error'].apply(lambda x: x.split(' ')[-1].replace(':','')).astype('int64')

In [16]:
df.groupby(df['message']).count()

,error,id
message,,
attempt to get argmax of an empty sequence,164,164


In [17]:
print("total attempted:", len(attempted))
print("number with exceptions:", len(df['message']))
print("fraction with exceptions:", len(df['message'])/len(attempted))

total attempted: 81239
number with exceptions: 187
fraction with exceptions: 0.0023018500966284668


In [18]:
merged = pd.merge(df, blendDF, left_on='id', right_on='id', how='left')

In [19]:
mask = merged['message'].apply(lambda x: x is None)
merged.loc[mask,'message'] = ''
merged['message_type'] = merged['message'].apply(lambda x: 'could not broadcast input array' if x.startswith('could') else x)
merged['outcome'] = merged['deblend_failed'].apply(lambda x: 'flagged failed' if x == True else ('not in catalog' if np.isnan(x) else 'not flagged'))
merged.groupby([merged.message_type, merged.outcome]).count()

error  message  \
message_type                               outcome                          
                                           flagged failed     19       19   
                                           not flagged         4        4   
attempt to get argmax of an empty sequence flagged failed    164      164   

                                                            id  coord_ra  \
message_type                               outcome                         
                                           flagged failed   19         0   
                                           not flagged       4         0   
attempt to get argmax of an empty sequence flagged failed  164         0   

                                                           coord_dec  parent  \
message_type                               outcome                             
                                           flagged failed          0      19   
                                           not flagged             0       4   
attempt to get argmax of an empty sequence flagged failed          0     164   

                                                           merge_footprint_i2  \
message_type                               outcome                              
                                           flagged failed                  19   
                                           not flagged                      4   
attempt to get argmax of an empty sequence flagged failed                 164   

                                                           merge_footprint_i  \
message_type                               outcome                             
                                           flagged failed                 19   
                                           not flagged                     4   
attempt to get argmax of an empty sequence flagged failed                164   

                                                           merge_footprint_r2  \
message_type                               outcome                              
                                           flagged failed                  19   
                                           not flagged                      4   
attempt to get argmax of an empty sequence flagged failed                 164   

                                                           merge_footprint_r  \
message_type                               outcome                             
                                           flagged failed                 19   
                                           not flagged                     4   
attempt to get argmax of an empty sequence flagged failed                164   

                                                           ...  \
message_type                               outcome         ...   
                                           flagged failed  ...   
                                           not flagged     ...   
attempt to get argmax of an empty sequence flagged failed  ...   

                                                           deblend_morphConvergenceFailed  \
message_type                               outcome                                          
                                           flagged failed                              19   
                                           not flagged                                  4   
attempt to get argmax of an empty sequence flagged failed                             164   

                                                           deblend_blendConvergenceFailed  \
message_type                               outcome                                          
                                           flagged failed                              19   
                                           not flagged                                  4   
attempt to get argmax of an empty sequence flagged failed                             164   

              

In [20]:
print("Some weird parents that logged 'Unable to deblend' but aren't in the catalog + flagged deblend_failed")
merged[(merged.outcome != "flagged failed")]

Some weird parents that logged 'Unable to deblend' but aren't in the catalog + flagged deblend_failed


,error,message,id,coord_ra,coord_dec,parent,merge_footprint_i2,merge_footprint_i,merge_footprint_r2,merge_footprint_r,...,deblend_morphConvergenceFailed,deblend_blendConvergenceFailed,deblend_edgePixels,deblend_failed,deblend_skipped,deblend_peak_center_x,deblend_peak_center_y,deblend_peak_instFlux,message_type,outcome
12,Unable to deblend source 43160302451163137:,,43160302451163137,NaN,NaN,0,False,True,False,True,...,False,False,False,False,False,NaN,NaN,NaN,,not flagged
13,Unable to deblend source 43160298156195841:,,43160298156195841,NaN,NaN,0,False,True,False,True,...,False,True,False,False,False,NaN,NaN,NaN,,not flagged
49,Unable to deblend source 43161170034556929:,,43161170034556929,NaN,NaN,0,False,True,False,True,...,False,True,False,False,False,NaN,NaN,NaN,,not flagged
92,Unable to deblend source 43162136402198529:,,43162136402198529,NaN,NaN,0,False,True,False,True,...,False,True,False,False,False,NaN,NaN,NaN,,not flagged


In [21]:
print("Count Exceptions")
merged['id'].apply(bool).groupby([merged.message_type, merged.outcome]).sum()

Count Exceptions


message_type                                outcome       
                                            flagged failed     19.0
                                            not flagged         4.0
attempt to get argmax of an empty sequence  flagged failed    164.0
Name: id, dtype: float64

In [22]:
print("Fraction Exceptions")
merged['id'].apply(bool).groupby([merged.message_type, merged.outcome]).sum()/len(attempted)

Fraction Exceptions


message_type                                outcome       
                                            flagged failed    0.000234
                                            not flagged       0.000049
attempt to get argmax of an empty sequence  flagged failed    0.002019
Name: id, dtype: float64

# Conclusions (End of Sprint)

* Exceptions reduced from 12% to 0.2% of blends over the sprint
* Some weird parents remain that were logged 'Unable to deblend' but are not flagged

# Appendix: Prototypes of each exception with how to reproduce 

Tracebacks and how to reproduce

## Selections from tiny5.o154139:

__could not broadcast input array from shape (43,43) into shape (43)__
```
161845 INFO  2019-05-14T10:04:17.068-0500 multiBandDriver.deblendCoaddSources: Read 195 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '26,28'}, tag=set())
161845 INFO  2019-05-14T10:04:04.366-0500 multiBandDriver: lsst-verify-worker01:161845: Start merge detections from [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '24,17'}, tag=set())]
161845 INFO  2019-05-14T10:25:05.731-0500 meas.deblender.deblend: id: 43161724085338251
161845 WARN  2019-05-14T10:25:06.021-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161724085338251: could not broadcast input array from shape (43,43) into shape (43)
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 213, in computePsfImage
    psfs = np.array(psfs)
ValueError: could not broadcast input array from shape (43,43) into shape (43)

Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 213, in computePsfImage
    psfs = np.array(psfs)

```

Reproduce with: `deblendCoaddSources.py  /datasets/hsc/repo --calib /datasets/hsc/repo/CALIB/ --rerun private/yusra/deblender-sprint/w_2019_18_05k:private/yusra/deblender-sprint/scratch/reproduce -c simultaneous=True multiBandDeblend.symmetric=False --id tract=9813 patch=26,28 filter=HSC-G^HSC-R^HSC-I^HSC-Z^HSC-Y^NB0921 --no-versions --longlog &> deblend26,28.log &`

---------------------

__'LsstSource' object has no attribute 'B'__


```
194805 INFO  2019-05-14T10:04:14.259-0500 multiBandDriver: lsst-verify-worker05:194805: Start deblending [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '25,8'}, tag=set())]
194805 INFO  2019-05-14T10:58:31.467-0500 meas.deblender.deblend: id: 43161500747038965
194805 WARN  2019-05-14T10:58:32.048-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161500747038965: 'LsstSource' object has no attribute 'B'
```

Reproduce with: `srun deblendCoaddSources.py /datasets/hsc/repo --calib /datasets/hsc/repo/CALIB/ --rerun private/yusra/deblender-sprint/w_2019_18_05k:private/yusra/deblender-sprint/scratch/reproduce -c simultaneous=True multiBandDeblend.symmetric=False --id tract=9813 patch=25,8 filter=HSC-G^HSC-R^HSC-I^HSC-Z^HSC-Y^NB0921 --no-versions --longlog`

--------------------------

__attempt to get argmax of an empty sequence__

```
80278 INFO  2019-05-14T10:04:17.516-0500 multiBandDriver.deblendCoaddSources: Read 223 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '27,9'}, tag=set())
80278 INFO  2019-05-14T10:04:17.516-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Deblending 223 sources in 6 exposure bands
80278 WARN  2019-05-14T10:04:18.012-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161779919912961: attempt to get argmax of an empty sequence
```

Reproduce with: `srun deblendCoaddSources.py /datasets/hsc/repo --calib /datasets/hsc/repo/CALIB/ --rerun private/yusra/deblender-sprint/w_2019_18_05k:private/yusra/deblender-sprint/scratch/reproduce -c simultaneous=True multiBandDeblend.symmetric=False --id tract=9813 patch=27,9 filter=HSC-G^HSC-R^HSC-I^HSC-Z^HSC-Y^NB0921 --no-versions --longlog`

Weirdly in reproduction we get through 43161779919912961, but same error on 43161779919912968. 

----------------------------
Selections from tiny4.o154140
---------------------------

__tuple index out of range__

*No traceback:*
```
42797 INFO  2019-05-14T10:46:12.141-0500 multiBandDriver.deblendCoaddSources: Read 255 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '22,18'}, tag=set())
42797 INFO  2019-05-14T10:46:12.154-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Deblending 255 sources in 6 exposure bands
42797 INFO  2019-05-14T10:47:47.464-0500 meas.deblender.deblend: id: 43161131379851276
42797 WARN  2019-05-14T10:47:47.807-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161131379851276: tuple index out of range
```

---

*`scarlet.source.SourceInitError: Zero or negative flux`*

```
42797 INFO  2019-05-14T10:46:10.123-0500 multiBandDriver: lsst-verify-worker23:42797: Start deblending [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '22,18'}, tag=set())]
42797 INFO  2019-05-14T10:46:12.141-0500 multiBandDriver.deblendCoaddSources: Read 255 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '22,18'}, tag=set())
42797 INFO  2019-05-14T10:46:12.154-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Deblending 255 sources in 6 exposure bands
42797 INFO  2019-05-14T10:48:30.233-0500 meas.deblender.deblend: id: 43161131379851349
42797 WARN  2019-05-14T10:48:30.536-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161131379851349: tuple index out of range
/project/yusra/lsst_devel/autograd/_eupspkg/binary/autograd/master-g6a9ef8c4bf/lib/python/autograd-1.2-py3.7.egg/autograd/numpy/fft.py:101: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return anp.pad(x, pads, 'constant')[slices]
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/source.py", line 22, in __init__
    symmetric, monotonic, center_step, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 331, in __init__
    thresh, True, monotonic)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 122, in init_extended_source
    sed = get_pixel_sed(sky_coord, scene, observations)  # amplitude is in sed
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 46, in get_pixel_sed
    raise SourceInitError(msg.format(*sky_coord))
scarlet.source.SourceInitError: Zero or negative flux at y=87, x=47

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 91, in deblend
    for center in peaks
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 91, in <listcomp>
    for center in peaks
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/source.py", line 27, in __init__
    center_step, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 253, in __init__
    super().__init__(self, sed, morph, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/component.py", line 85, in __init__
    self.B = sed.shape[0]
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/component.py", line 110, in shape
    return (self.B, self.Ny, self.Nx)
AttributeError: 'LsstSource' object has no attribute 'B'
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)

    
```

---
    
```
 42797 WARN  2019-05-14T10:48:39.404-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161131379851373: tuple index out of range
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
```

`srun deblendCoaddSources.py  /datasets/hsc/repo --calib /datasets/hsc/repo/CALIB/  --rerun private/yusra/deblender-sprint/w_2019_18_05k:private/yusra/deblender-sprint/scratch/reproduce -c  simultaneous=True multiBandDeblend.symmetric=False --id tract=9813 patch=23,19 filter=HSC-G^HSC-R^HSC-I^HSC-Z^HSC-Y^NB0921 --no-versions --longlog`
`

Still working on getting a SourceInitError in a single-patch non-mulitbandDriver setting. 